In [1]:
import requests as rq
from bs4 import BeautifulSoup
import json
import csv

In [2]:
#fetching the website
def get_link(link):
    web = rq.get(link)
    #print(web)
    soup = BeautifulSoup(web.content , "html.parser")
    #print(soup)
    return soup

In [3]:
def scrape(link , country):
    
    soup=get_link(link)

    desc=['Name',  'Batting' , 'Bowling','Age','Position' , 'Team']
    b=soup.find_all('div' , class_="ds-relative ds-flex ds-flex-row ds-space-x-4 ds-p-3")
    data=[]
    number=0

    for i in b:
        row_data={}
        for j in i.find_all('span' , class_='ds-text-compact-s ds-font-bold ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer'):
            row_data[desc[0]]=j.get_text()
        for j in (i.find_all("div" , class_="ds-flex ds-items-center ds-space-x-1")):
            for number,k in enumerate((j.find_all('span' , class_='ds-text-compact-xxs ds-font-bold'))):
                row_data[desc[number+3]] = k.get_text()
        for j in i.find_all('div' , class_="ds-flex ds-items-start ds-space-x-1"):
            for k in (j.find_all('span' , class_='ds-text-compact-xxs ds-font-bold')):
                row_data[desc[(number%2)+1]] = k.get_text()
                number+=1
        for j in i.find_all("p" ,class_="ds-text-tight-s ds-font-regular ds-mb-2 ds-mt-1"):
            row_data[desc[4]]=j.get_text()
        
        row_data[desc[5]] = country
        data.append(row_data)
    
    info.append(data)
        

In [4]:
info = []
links = {}  #manually adding the links to the squad info
links.update({'New Zealand':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/new-zealand-squad-1397557/series-squads"})
links.update({'Afghanistan':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/afghanistan-squad-1398042/series-squads"})
links.update({'Australia':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/australia-squad-1396545/series-squads"})
links.update({'Bangladesh':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/bangladesh-squad-1400121/series-squads"})
links.update({'England':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/england-squad-1396558/series-squads"})
links.update({'India':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-squad-1396344/series-squads"})
links.update({'Netherlands':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/netherlands-squad-1396753/series-squads"})
links.update({'Pakistan':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/pakistan-squad-1399493/series-squads"})
links.update({'South Africa':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/south-africa-squad-1396372/series-squads"})
links.update({'Sri Lanka':"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/sri-lanka-squad-1400061/series-squads"})

In [5]:
#for getting information pertaining to every player
for i in links:
    scrape(links[i] , i)
    
with open(r"C:\Users\Ram\Desktop\info.json", "w") as json_file:
        json.dump(info, json_file, indent=4)

In [6]:
#for getting the links to each player's profile
url = 'https://www.sportskeeda.com/cricket/cricket-world-cup-team-squad'
soup = get_link(url)
links=[]
names=[]
text = soup.find_all("table")

for table in text:
    
    for i , rows in enumerate(table.find_all('tr')): 
        
        if(i==0):
            continue
        name=rows.find('a').get_text()
        link=("https://www.sportskeeda.com" + rows.find('a').get('href'))
        links.append(link)
        names.append(name)

In [7]:
#for getting the player image links from the player's profile
updated_links = []

for link in links:
    soup = get_link(link)
    img_tag = soup.find('img', width='134', height='134')

    fallbacks_data = img_tag.get('data-img-fallbacks')

    # Splitting the values by pipe (|) separator and get the HTTPS link
    https_link = fallbacks_data.split('|')[0]

    updated_links.append(https_link)

In [8]:
#storing all the player and url pairs to an array
final=[]
for i in range(len(names)):
    dict={}
    dict.update({"name":f"{names[i]}"})
    dict.update({"link":f"{links[i]}"})
    final.append(dict)

In [9]:
headers = final[0].keys()#appending the headers of the file

#CSV file
with open('data.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(final)